# Exciton Group Theory Analysis Example

This notebook demonstrates the use of the `ExcitonGroupTheory` class for analyzing the symmetry properties of exciton states in crystalline materials using group theory.

## Theoretical Background

### Group Theory for Excitons

Exciton states transform according to the irreducible representations of the little group $G_k$ of the exciton momentum $\mathbf{k}$. The symmetry analysis involves:

1. **Little Group Identification**: Finding all symmetry operations that leave the exciton momentum invariant
2. **Representation Matrix Calculation**: Computing how exciton states transform under symmetry operations
3. **Character Analysis**: Determining the trace of representation matrices
4. **Irreducible Representation Decomposition**: Classifying states according to their symmetry properties

### Mathematical Formalism

The representation matrix for symmetry operation $R$ is:

$D^{(n)}_R = \langle\psi_n(R\mathbf{k})| U(R) |\psi_n(\mathbf{k})\rangle$

The character of this representation:

$\chi^{(n)}(R) = \text{Tr}[D^{(n)}_R]$

Irreducible representation decomposition using the reduction formula:
$a_i = \frac{1}{|G|} \sum_{R \in G} \chi^{(R)} \chi_i^{(R)*}$

## What this notebook demonstrates:

1. How to initialize the ExcitonGroupTheory class
2. How to perform group theory analysis for exciton states
3. How to interpret symmetry results and optical selection rules
4. How to save and visualize the analysis results

**Note**: Make sure you have the required Yambo database files in the appropriate directories before running this example.

## Setup and Imports

First, let's import the necessary modules and set up our environment:

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

from yambopy.optical_properties.exciton_group_theory import ExcitonGroupTheory

# Set up plotting parameters
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## Configuration Parameters

Set up the basic input parameters for your calculation. Adjust these paths and parameters according to your specific calculation setup:

In [ ]:
# Basic input parameters
# Adjust these paths according to your calculation setup
path = './'  # Current directory or path to your calculation
SAVE_dir = './SAVE'
BSE_dir = './GW_BSE/bse'  # or 'GW_BSE' depending on your setup
LELPH_dir = './lelph'  # Directory containing electron-phonon data

# Exciton analysis parameters
iQ = 1  # Exciton Q-point index (1-based, as in Yambo)
nstates = 10  # Number of exciton states to analyze
degen_thres = 0.001  # Degeneracy threshold in eV

# Band range (adjust according to your calculation)
bands_range = [1, 20]  # Example: bands 1 to 20

print("Configuration:")
print(f"Path: {path}")
print(f"BSE directory: {BSE_dir}")
print(f"LELPH directory: {LELPH_dir}")
print(f"Analyzing Q-point: {iQ}")
print(f"Number of states: {nstates}")
print("-" * 50)

## Initialize ExcitonGroupTheory Class

The `ExcitonGroupTheory` class reads the necessary database files and sets up the symmetry operations. The initialization process involves:

1. **Database Reading**: Loading lattice, wavefunction, BSE, and electron-phonon databases
2. **Symmetry Setup**: Reading crystallographic symmetry operations
3. **D-matrix Preparation**: Setting up wavefunction rotation matrices
4. **K-point Mapping**: Building k-point search trees for efficient operations

In [ ]:
# Initialize the ExcitonGroupTheory class
print("Initializing ExcitonGroupTheory class...")
print("This will read the database files and set up symmetry operations.")

egt = ExcitonGroupTheory(
    path=path,
    save='SAVE',
    BSE_dir=BSE_dir,
    LELPH_dir=LELPH_dir,
    bands_range=bands_range,
    read_symm_from_ns_db_file=True  # Read symmetries from ns.db1
)

print("\nInitialization completed successfully!")
print(f"Number of symmetry operations: {len(egt.symm_mats)}")
print(f"Number of IBZ k-points: {egt.nibz}")
print(f"Number of bands: {egt.nbands}")
print(f"Lattice vectors shape: {egt.lat_vecs.shape}")
print(f"Reciprocal lattice vectors shape: {egt.blat_vecs.shape}")

## Perform Group Theory Analysis

The `analyze_exciton_symmetry()` method performs the complete group theory analysis:

### Method Overview

1. **`_determine_little_group()`**: Identifies symmetry operations that leave the Q-point invariant
2. **`_read_exciton_states()`**: Loads BSE eigenvalues and eigenvectors
3. **`_group_degenerate_states()`**: Groups states by energy within the degeneracy threshold
4. **`_compute_representation_matrices()`**: Calculates how states transform under symmetry
5. **`_compute_characters()`**: Computes traces of representation matrices
6. **`_decompose_representations()`**: Decomposes into irreducible representations
7. **`_analyze_optical_activity()`**: Determines optical selection rules

In [ ]:
# Perform the group theory analysis
print(f"Performing group theory analysis for Q-point {iQ}...")
print(f"Analyzing {nstates} exciton states with degeneracy threshold {degen_thres} eV")

results = egt.analyze_exciton_symmetry(
    iQ=iQ,
    nstates=nstates,
    degen_thres=degen_thres
)

print("\nAnalysis completed successfully!")
print("\n" + "=" * 60)
print("GROUP THEORY ANALYSIS RESULTS")
print("=" * 60)

# Display basic information
print(f"Q-point coordinates: {results['q_point']}")
print(f"Point group: {results['point_group_label']}")
print(f"Little group size: {len(results['little_group'])}")
print(f"Number of energy levels: {len(results['unique_energies'])}")
print(f"Total exciton states analyzed: {len(results['exciton_energies'])}")

## Detailed Results Analysis

Let's examine the detailed results of the group theory analysis:

In [ ]:
print("\n" + "=" * 60)
print("ENERGY LEVELS AND SYMMETRY CLASSIFICATION")
print("=" * 60)
print(f"{'Level':<6} {'Energy (eV)':<12} {'Degeneracy':<11} {'Irrep':<20} {'Optical Activity'}")
print("-" * 80)

for i, (energy, degen, irrep, activity) in enumerate(zip(
    results['unique_energies'],
    results['degeneracies'],
    results['irrep_decomposition'],
    results['optical_activity']
)):
    # Determine optical activity status
    status = []
    if activity['electric_dipole_allowed']:
        status.append('Optical')
    if activity['raman_active']:
        status.append('Raman')
    if activity['ir_active']:
        status.append('IR')
    
    status_str = ', '.join(status) if status else 'Forbidden'
    
    print(f"{i+1:<6} {energy:<12.4f} {degen:<11} {irrep:<20} {status_str}")

# Display symmetry classes
print("\n" + "=" * 60)
print("SYMMETRY CLASSES")
print("=" * 60)
for i, class_name in enumerate(results['classes']):
    operations = results['class_dict'].get(i, [])
    print(f"{class_name:<10}: {operations}")

## Optical Selection Rules Analysis

The optical activity analysis determines which transitions are allowed based on symmetry selection rules:

In [ ]:
print("\n" + "=" * 60)
print("OPTICAL SELECTION RULES ANALYSIS")
print("=" * 60)

for i, (energy, irrep, activity) in enumerate(zip(
    results['unique_energies'][:5],  # Show first 5 states
    results['irrep_decomposition'][:5],
    results['optical_activity'][:5]
)):
    print(f"\nExciton State {i+1}: {energy:.4f} eV")
    print(f"Irreducible Representation: {irrep}")
    print("-" * 40)
    
    # Selection rules
    print("Selection Rules:")
    print(f"  Electric Dipole Allowed: {activity['electric_dipole_allowed']}")
    print(f"  Raman Active: {activity['raman_active']}")
    print(f"  IR Active: {activity['ir_active']}")
    
    # Physical interpretation
    print("\nPhysical Interpretation:")
    if activity['electric_dipole_allowed']:
        print("  ✓ Bright exciton - observable in absorption/photoluminescence")
        print("  ✓ Can couple to light via electric dipole transitions")
    else:
        print("  ✗ Dark exciton - forbidden in electric dipole approximation")
        print("  ✗ Not directly observable in linear optical spectroscopy")
    
    if activity['raman_active']:
        print("  ✓ Observable in Raman scattering experiments")
    
    if activity['ir_active']:
        print("  ✓ Observable in infrared spectroscopy")
    
    # Show detailed rules if available
    if activity['selection_rules']:
        print("\nDetailed Symmetry Rules:")
        for j, rule in enumerate(activity['selection_rules']):
            if rule.get('notes'):
                for note in rule['notes']:
                    print(f"  • {note}")

## Visualization of Results

Let's create visualizations to better understand the symmetry analysis results:

In [ ]:
# Create comprehensive visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Energy level diagram
energies = results['unique_energies']
degeneracies = results['degeneracies']
irreps = results['irrep_decomposition']
activities = results['optical_activity']

# Plot energy levels with color coding for optical activity
for i, (energy, degen, irrep, activity) in enumerate(zip(energies, degeneracies, irreps, activities)):
    if activity['electric_dipole_allowed']:
        color = 'red'
        label = 'Bright (Optically Active)' if i == 0 else ''
    else:
        color = 'blue'
        label = 'Dark (Optically Forbidden)' if i == 0 else ''
    
    # Draw energy level
    ax1.hlines(energy, 0, 1, colors=color, linewidth=4, label=label)
    
    # Add labels
    ax1.text(1.1, energy, f'{irrep} (deg={degen})', 
             verticalalignment='center', fontsize=10)

ax1.set_xlim(-0.1, 2.5)
ax1.set_ylabel('Energy (eV)')
ax1.set_title('Exciton Energy Levels and Symmetries')
ax1.legend()
ax1.set_xticks([])
ax1.grid(True, alpha=0.3)

# 2. Optical activity summary
activity_counts = {'Optical': 0, 'Raman': 0, 'IR': 0, 'Forbidden': 0}

for activity in activities:
    if activity['electric_dipole_allowed']:
        activity_counts['Optical'] += 1
    if activity['raman_active']:
        activity_counts['Raman'] += 1
    if activity['ir_active']:
        activity_counts['IR'] += 1
    if not any([activity['electric_dipole_allowed'], 
                activity['raman_active'], 
                activity['ir_active']]):
        activity_counts['Forbidden'] += 1

activities_list = list(activity_counts.keys())
counts = list(activity_counts.values())
colors = ['red', 'green', 'orange', 'gray']

bars = ax2.bar(activities_list, counts, color=colors, alpha=0.7)
ax2.set_ylabel('Number of States')
ax2.set_title('Optical Activity Summary')
ax2.set_ylim(0, max(counts) + 1)
ax2.grid(True, alpha=0.3)

# Add value labels on bars
for bar, count in zip(bars, counts):
    if count > 0:
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                str(count), ha='center', va='bottom')

# 3. Degeneracy distribution
unique_degens, degen_counts = np.unique(degeneracies, return_counts=True)
ax3.bar(unique_degens, degen_counts, alpha=0.7, color='purple')
ax3.set_xlabel('Degeneracy')
ax3.set_ylabel('Number of Levels')
ax3.set_title('Degeneracy Distribution')
ax3.grid(True, alpha=0.3)

# 4. Energy distribution histogram
all_energies = results['exciton_energies']
ax4.hist(all_energies, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
ax4.axvline(np.mean(all_energies), color='red', linestyle='--', 
           label=f'Mean: {np.mean(all_energies):.3f} eV')
ax4.set_xlabel('Energy (eV)')
ax4.set_ylabel('Number of States')
ax4.set_title('Exciton Energy Distribution')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Save Analysis Results

The `save_analysis_results()` method saves the complete analysis to a text file for future reference:

In [ ]:
# Save the analysis results
output_filename = f'exciton_symmetry_Q{iQ}_analysis.txt'

print(f"Saving analysis results to: {output_filename}")
egt.save_analysis_results(results, output_filename)

print("\nResults saved successfully!")
print(f"\nFile contains:")
print("- Point group identification")
print("- Little group operations")
print("- Symmetry classes")
print("- Energy levels with irreducible representations")
print("- Optical selection rules")

# Display a preview of the saved file
if os.path.exists(output_filename):
    print(f"\nPreview of {output_filename}:")
    print("-" * 50)
    with open(output_filename, 'r') as f:
        lines = f.readlines()[:20]  # Show first 20 lines
        for line in lines:
            print(line.rstrip())
    print("...")
    print(f"(showing first 20 lines of {len(open(output_filename).readlines())} total lines)")

## Summary and Physical Interpretation

Let's summarize the key findings from our group theory analysis:

In [ ]:
print("\n" + "=" * 70)
print("SUMMARY AND PHYSICAL INTERPRETATION")
print("=" * 70)

# System information
print(f"\n🔬 SYSTEM ANALYSIS:")
print(f"Point Group: {results['point_group_label']}")
print(f"Q-point: {results['q_point']}")
print(f"Little Group Size: {len(results['little_group'])} operations")
print(f"Symmetry Classes: {len(results['classes'])}")

# Energy analysis
print(f"\n📊 ENERGY LEVEL ANALYSIS:")
print(f"Total States Analyzed: {len(results['exciton_energies'])}")
print(f"Unique Energy Levels: {len(results['unique_energies'])}")
print(f"Energy Range: {np.min(results['exciton_energies']):.3f} - {np.max(results['exciton_energies']):.3f} eV")
print(f"Average Degeneracy: {np.mean(results['degeneracies']):.1f}")

# Optical properties
bright_states = sum(1 for activity in results['optical_activity'] 
                   if activity['electric_dipole_allowed'])
dark_states = len(results['optical_activity']) - bright_states
raman_states = sum(1 for activity in results['optical_activity'] 
                  if activity['raman_active'])
ir_states = sum(1 for activity in results['optical_activity'] 
               if activity['ir_active'])

print(f"\n🌟 OPTICAL PROPERTIES:")
print(f"Bright Excitons (Electric Dipole Allowed): {bright_states}/{len(results['optical_activity'])}")
print(f"Dark Excitons (Optically Forbidden): {dark_states}/{len(results['optical_activity'])}")
print(f"Raman Active States: {raman_states}/{len(results['optical_activity'])}")
print(f"IR Active States: {ir_states}/{len(results['optical_activity'])}")

# Experimental predictions
print(f"\n🔬 EXPERIMENTAL PREDICTIONS:")
if bright_states > 0:
    print(f"✓ Absorption spectroscopy: Should observe {bright_states} bright exciton(s)")
    print(f"✓ Photoluminescence: Emission expected from optically active states")
else:
    print(f"✗ All excitons are dark - no direct optical transitions expected")

if raman_states > 0:
    print(f"✓ Raman spectroscopy: {raman_states} Raman-active mode(s) expected")

if ir_states > 0:
    print(f"✓ Infrared spectroscopy: {ir_states} IR-active mode(s) expected")

# Symmetry insights
print(f"\n🔍 SYMMETRY INSIGHTS:")
if results['point_group_label'] in ['C6v', 'D6h', '6/m', 'D3h']:
    print(f"✓ Hexagonal/trigonal symmetry detected")
    print(f"✓ Expect in-plane polarized optical transitions")
    print(f"✓ Possible doubly degenerate E-type states")

if any(deg == 2 for deg in results['degeneracies']):
    print(f"✓ Doubly degenerate states found (E-type irreps)")

if any(deg == 3 for deg in results['degeneracies']):
    print(f"✓ Triply degenerate states found (T-type irreps)")

print(f"\n" + "=" * 70)
print(f"GROUP THEORY ANALYSIS COMPLETED SUCCESSFULLY")
print(f"=" * 70)

## Conclusion

This example demonstrated the comprehensive capabilities of the `ExcitonGroupTheory` class for analyzing the symmetry properties of exciton states. The analysis provides:

### Key Results:
1. **Point Group Identification**: Automatic detection of crystallographic symmetries
2. **Irreducible Representation Analysis**: Classification of exciton states by symmetry
3. **Optical Selection Rules**: Prediction of allowed optical transitions
4. **Physical Interpretation**: Clear distinction between bright and dark excitons

### Applications:
- **Experimental Design**: Choosing appropriate spectroscopic techniques
- **Material Characterization**: Understanding symmetry-breaking effects
- **Theoretical Validation**: Comparing with experimental observations
- **Property Prediction**: Forecasting optical behavior of new materials

### Method Functionality:
- **`analyze_exciton_symmetry()`**: Main analysis method performing complete group theory analysis
- **`save_analysis_results()`**: Saves comprehensive results to text files
- **Optical Activity Analysis**: Determines electric dipole, Raman, and IR activity
- **Visualization Support**: Results compatible with matplotlib for plotting

The group theory analysis provides fundamental insights into exciton physics that complement and enhance traditional electronic structure calculations, enabling deeper understanding of optical properties in crystalline materials.